<a href="https://colab.research.google.com/github/GustaFTW/TensorFlow-ZTM/blob/main/section8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language (could be sequences of text or speech)

Another common term for NLP problems is sequence to sequence problems (seq2seq).

## Check for GPU

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-fe65cd04-062e-b58f-2eff-d0015a5b74e3)


## Get helper functions

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-01-20 14:08:00--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-01-20 14:08:00 (100 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaster os not disaster).

See original source: https://www.kaggle.com/competitions/nlp-getting-started/data

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-01-20 14:08:07--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.62.128, 172.253.115.128, 172.253.122.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.62.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.008s  

2023-01-20 14:08:07 (76.1 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualize our text samples, we first have to read them in, one way to do so would be to use Python


But I prefer to get visual straight away.

So another way to do this is to use pandas...

In [ ]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=69)
train_df_shuffled.head()

,id,keyword,location,text,target
421,611,arsonist,[ Blonde Bi Fry. ],'wHeRE's mY aRsOnISt aT???',0
5047,7195,natural%20disaster,home,she's a natural disaster she's the last of the...,0
5046,7194,mudslide,NaN,@nikistitz even the one that looked like a mud...,0
1881,2702,crush,NaN,kenny holland crush da vida,0
6543,9362,survived,NaN,Not one character in the final destination ser...,0


In [ ]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
# How many total samples?
len(train_df) + len(test_df)

10876

In [ ]:
# Let's visualize some random training examples
import random
number_samples = 3
random_index = random.randint(0, len(train_df)-number_samples)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+number_samples].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}\n")
    print("---\n")

Target: 1 (real disaster)
Text:
Man Currensy really be talkin that talk... I'd be more devastated if he had a ghostwriter than anybody else....

---

Target: 0 (not real disaster)
Text:
Ugh So hungry I'm going to demolish this food!

---

Target: 1 (real disaster)
Text:
Obama Declares Disaster for Typhoon-Devastated Saipan: Obama signs disaster declaration for Northern Marians a... http://t.co/BHZr9UgUs2

---



### Split data into training and validation sets

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # use 10% of training data for validation split
                                                                            random_state=69)

In [ ]:
# Check the  lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [ ]:
# Check the first 10 examples
train_sentences[:10], train_labels[:10]

(array(['@ChrisDanielShow Nearly-Fatal Pee in San Francisco http://t.co/1tvlFrhm1m',
        "babe I'm gonna ruin you if you let me stay",
        "We're happily collided :)",
        "RT THR 'RT THRArchives: 1928: When Leo the MGM Lion Survived a Plane Crash #TBT http://t.co/Wpkl2qNiQW http://t.co/BD52FxDvhQ'",
        '@FoxNews let me report it to u people instead Mr.Obama just declares CNMI federal disaster area post typhoon soudelor. U guys 2slow2report.',
        "Don't say @ALIPAPER: women got problems this #keepingtheviginaclean thing is very interesting n less expensive #metrofmtalk'",
        "'Jet fuel cant melt steel'\n'The structural failure is illogical'\n'The second plane crashing into the building is fake'\n'It was a bomb'",
        'A grade in Black Horse Famine[MEGA]. Score 0840728\nhttp://t.co/pdiit0AF3Q\n#Dynamix http://t.co/ZQ5KDOx7BY',
        'Even when I was a kid haha super late but folks used to bash me for that shit I understand he survived cancer but he still

## Converting text into numbers

When dealing with a text problem, one of the first trhings you'll have to do before you can build a model is to converty your text to number.

there are a few ways to do this, namely:
* Tokenziation - directo mapping of token (a token could be a word or a character) to number
* Embedding - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

### Text vectorization (tokenization)

In [ ]:
train_sentences[:10]

array(['@ChrisDanielShow Nearly-Fatal Pee in San Francisco http://t.co/1tvlFrhm1m',
       "babe I'm gonna ruin you if you let me stay",
       "We're happily collided :)",
       "RT THR 'RT THRArchives: 1928: When Leo the MGM Lion Survived a Plane Crash #TBT http://t.co/Wpkl2qNiQW http://t.co/BD52FxDvhQ'",
       '@FoxNews let me report it to u people instead Mr.Obama just declares CNMI federal disaster area post typhoon soudelor. U guys 2slow2report.',
       "Don't say @ALIPAPER: women got problems this #keepingtheviginaclean thing is very interesting n less expensive #metrofmtalk'",
       "'Jet fuel cant melt steel'\n'The structural failure is illogical'\n'The second plane crashing into the building is fake'\n'It was a bomb'",
       'A grade in Black Horse Famine[MEGA]. Score 0840728\nhttp://t.co/pdiit0AF3Q\n#Dynamix http://t.co/ZQ5KDOx7BY',
       'Even when I was a kid haha super late but folks used to bash me for that shit I understand he survived cancer but he still cheated'

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=10000, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words
                                    output_mode="int", # how to map tokens into numbers
                                    output_sequence_length=None, # how long do you want your sequences to be?
                                    pad_to_max_tokens=True)

In [ ]:
# Find the average number of tokens (words) in the trianing tweets
round(sum([len(i.split()) for i in train_sentences])/ len(train_sentences))

15

In [ ]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does a model see)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [ ]:
 # Fit the text vectorizer to the training text 
 text_vectorizer.adapt(train_sentences)

In [ ]:
train_sentences[:10]

array(['@ChrisDanielShow Nearly-Fatal Pee in San Francisco http://t.co/1tvlFrhm1m',
       "babe I'm gonna ruin you if you let me stay",
       "We're happily collided :)",
       "RT THR 'RT THRArchives: 1928: When Leo the MGM Lion Survived a Plane Crash #TBT http://t.co/Wpkl2qNiQW http://t.co/BD52FxDvhQ'",
       '@FoxNews let me report it to u people instead Mr.Obama just declares CNMI federal disaster area post typhoon soudelor. U guys 2slow2report.',
       "Don't say @ALIPAPER: women got problems this #keepingtheviginaclean thing is very interesting n less expensive #metrofmtalk'",
       "'Jet fuel cant melt steel'\n'The structural failure is illogical'\n'The second plane crashing into the building is fake'\n'It was a bomb'",
       'A grade in Black Horse Famine[MEGA]. Score 0840728\nhttp://t.co/pdiit0AF3Q\n#Dynamix http://t.co/ZQ5KDOx7BY',
       'Even when I was a kid haha super late but folks used to bash me for that shit I understand he survived cancer but he still cheated'

In [ ]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[257,   3, 239,   5,  13, 702,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
        \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
The cool kids asked me if I wanted to hang out after school so I had a panic attack and had to go to the hospital #autismawareness        

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   2,  526,  516, 1735,   26,   45,    8, 1028,    4, 2994,   36,
          41,  179,   27,    8]])>

In [ ]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
words_in_vocab[:5] # get the most common words
words_in_vocab[-5:] # get the least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {words_in_vocab[:10]}")
print(f"5 least common words: {words_in_vocab[-5:]}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'to', 'in', 'of', 'and', 'i', 'is']
5 least common words: ['panicked', 'pandora', 'pandemoniumiso', 'pandemic', 'panda']


### Creating an Embedding using an Embedding layer

To make our embedding, we're going to use TensorFlow's embedding layer

The parameters we care most about for our embedding layer:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented as a vector 100 long
* `input_length` = length of the sequences being passed to the embedding layer

In [ ]:
embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # output shape,
                             embeddings_initializer="uniform",
                             input_length=max_length # how long is each input
                             )

embedding

In [ ]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nEmbedded version:")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 I was in a horrible car accident this past Sunday. I'm finally able to get around. Thank you GOD??        

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00958958, -0.01776352,  0.02393601, ...,  0.02883032,
          0.04363749, -0.02646998],
        [ 0.01479634,  0.02544777,  0.04169513, ...,  0.02023281,
          0.01929759,  0.01687864],
        [ 0.02153952,  0.04665888,  0.00413417, ..., -0.01364823,
          0.00702504, -0.00481465],
        ...,
        [ 0.04010587,  0.03562632, -0.03427448, ...,  0.02622775,
         -0.03557865, -0.03935429],
        [-0.02902803,  0.04448882,  0.0252008 , ...,  0.04356781,
         -0.03129878,  0.01464598],
        [-0.01849232,  0.03950007, -0.02708852, ..., -0.00439485,
          0.01647099,  0.01514309]]], dtype=float32)>

In [ ]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.00958958, -0.01776352,  0.02393601,  0.00878153,  0.0195506 ,
        -0.00229176, -0.00478513, -0.02452946,  0.00194732,  0.01259555,
         0.00474857, -0.02624936,  0.02237291, -0.02633213, -0.0396691 ,
         0.04433174, -0.02575672,  0.03926647, -0.03960979, -0.02139329,
        -0.0408328 ,  0.04421889, -0.03518897, -0.03560494, -0.00259489,
         0.04302828,  0.02674792, -0.01832313,  0.01587128, -0.01029665,
         0.01948318, -0.02689917, -0.04281396,  0.01000056, -0.0488076 ,
         0.00956049, -0.03409134, -0.04747262,  0.00970721, -0.03773642,
        -0.02471132, -0.00240294, -0.00766774,  0.04968861, -0.04410336,
         0.02768162,  0.00922175, -0.01874187, -0.04762753,  0.03857002,
         0.04166759,  0.02272904,  0.0415164 ,  0.02033942,  0.00637997,
         0.04268363,  0.02369162, -0.01579813,  0.04327616,  0.04946462,
         0.03028542, -0.03485493,  0.04363028,  0.00844611,  0.0259507 ,
  

## Modelling a text dataset (running a series of experiments)

Now we've got a way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTMmodel (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: TensorFlow Hub Pretrained Featre Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

How are we going to approach all of these?

Use the standard steps in modelling with tensorflow:

* Create a model
* Build a model
* Fit a model
* Evaluate our model

### Model 0: Getting a baseline

As with all machine learning modelling experiments, it's important to create a baseline model so you've got a benchmark for future experiments to build upon.

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.

> **Note:** It's common practice to use non-DL algorithms as a baseline because of their spped and then later using DL to see if you can improve upon them.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")


Our baseline model achieves an accuracy of: 79.79%


In [ ]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

### Creating an evaluation function for our model experiments

Let's make a function to evaluate our modelling experiment predictions using:

* Accuracy
* Precision
* Recall
* F1-score

In [ ]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
    """
    Calculates model accuracy, precision, recall and f1 score of a binary classification model.
    """

    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    # Calculate model precision, recall and f1-score using "weighted" average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    model_results = {"accuracy": model_accuracy,
                     "precision": model_precision,
                     "recall": model_recall,
                     "f1": model_f1}
    return model_results

In [ ]:
# Get baseline results
baseline_results = calculate_results(val_labels, baseline_preds)
baseline_results

{'accuracy': 79.79002624671917,
 'precision': 0.8049118114708822,
 'recall': 0.7979002624671916,
 'f1': 0.7932923017133588}

### Model 1: A simple dense model

In [ ]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save Tensorboard logs
SAVE_DIR = "model_logs"

In [ ]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers 
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [ ]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model
history_1 = model_1.fit(train_sentences,
                        train_labels, 
                        epochs=5,
                        validation_data=(val_sentences, val_labels),
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20230120-150048
Epoch 1/5
215/215 [==============================] - 4s 6ms/step - loss: 0.6119 - accuracy: 0.6919 - val_loss: 0.5313 - val_accuracy: 0.7598
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.4417 - accuracy: 0.8162 - val_loss: 0.4707 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3470 - accuracy: 0.8610 - val_loss: 0.4596 - val_accuracy: 0.7940
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2829 - accuracy: 0.8902 - val_loss: 0.4693 - val_accuracy: 0.7913
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2355 - accuracy: 0.9120 - val_loss: 0.4889 - val_accuracy: 0.7782


In [ ]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 9ms/step - loss: 0.4889 - accuracy: 0.7782


[0.4889427423477173, 0.778215229511261]

In [ ]:
# Make some predictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [ ]:
# Look at a single prediction
model_1_pred_probs[0]

array([0.94225055], dtype=float32)

In [ ]:
# Look at the first 10
model_1_pred_probs[:10]

array([[0.94225055],
       [0.5889235 ],
       [0.04023167],
       [0.02157172],
       [0.1572312 ],
       [0.96717256],
       [0.97165316],
       [0.03744039],
       [0.12288823],
       [0.99855274]], dtype=float32)

In [ ]:
# Convert model prediction probabilities to label format

model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)>

In [ ]:
# calculate our model_1 reulsts
model_1_results = calculate_results(val_labels,
                                    model_1_preds)
model_1_results

{'accuracy': 77.82152230971128,
 'precision': 0.7775276018850527,
 'recall': 0.7782152230971129,
 'f1': 0.7775654664384112}

In [ ]:
baseline_results

{'accuracy': 79.79002624671917,
 'precision': 0.8049118114708822,
 'recall': 0.7979002624671916,
 'f1': 0.7932923017133588}

In [ ]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])